# SQL & Python

## Exemple avec SQLite

### Connexion à la base de données

Pour utiliser SQLite avec Python, on crée une connexion avec une base de données. 
Si elle n'existe pas elle est crée

In [43]:
import sqlite3

# Connexion à la base de données (si elle n'existe pas, elle sera créée)
conn = sqlite3.connect('data/ma_base_de_donnees.db')

### Création d'une table

Définissons une table simple pour stocker des données.

In [44]:
# Création d'un curseur pour exécuter des commandes SQL
cur = conn.cursor()

# Définition du schéma de la table
cur.execute('''
    CREATE TABLE IF NOT EXISTS utilisateurs (
        id INTEGER PRIMARY KEY,
        nom TEXT,
        age INTEGER
    )
''')

# Validation des modifications
conn.commit()

### Opérations CRUD (Create, Read, Update, Delete)

#### Insertion de données

Ajoutons un nouvel utilisateur à la table.

In [45]:
cur.execute("INSERT INTO utilisateurs (nom, age) VALUES (?, ?)", ('Alice', 25))
conn.commit()

#### Insertion de données par batch

In [46]:
data_to_insert = [
    ('Alice', 25),
    ('Bob', 30),
    ('Charlie', 22),
    ('David', 28)
]
cur.executemany("INSERT INTO utilisateurs (nom, age) VALUES (?, ?)", data_to_insert)
conn.commit()

#### Lecture des données

Récupérons toutes les données de la table.

In [47]:
cur.execute("SELECT * FROM utilisateurs")
resultats = cur.fetchall()

for row in resultats:
    print(row)

(3, 'Bob', 30)
(4, 'Charlie', 22)
(5, 'David', 28)
(7, 'Bob', 30)
(8, 'Charlie', 22)
(9, 'David', 28)
(12, 'Bob', 30)
(13, 'Charlie', 22)
(14, 'David', 28)
(17, 'Bob', 30)
(18, 'Charlie', 22)
(19, 'David', 28)
(22, 'Bob', 30)
(23, 'Charlie', 22)
(24, 'David', 28)
(25, 'Alice', 25)
(26, 'Alice', 25)
(27, 'Bob', 30)
(28, 'Charlie', 22)
(29, 'David', 28)


#### Mise à jour de données

Mettons à jour l'âge de l'utilisateur Alice.

In [48]:
cur.execute("UPDATE utilisateurs SET age=? WHERE nom=?", (26, 'Alice'))
conn.commit()

#### Suppression de données

Supprimons l'utilisateur Alice de la table.

In [49]:
cur.execute("DELETE FROM utilisateurs WHERE nom=?", ('Alice',))
conn.commit()

### Fonction d'aggrégations

In [50]:
# Exemple d'utilisation des fonctions d'agrégation
cur.execute("SELECT COUNT(*) FROM utilisateurs")
total_users = cur.fetchone()[0]
print(f"Nombre total d'utilisateurs : {total_users}")

cur.execute("SELECT AVG(age) FROM utilisateurs")
average_age = cur.fetchone()[0]
print(f"Âge moyen des utilisateurs : {average_age:.2f}")



Nombre total d'utilisateurs : 18
Âge moyen des utilisateurs : 26.67


In [51]:
# Fermeture de la connexion
#conn.close()

### Aspects avancés

#### Transactions
Les transactions permettent d'assurer l'intégrité des données en garantissant que plusieurs opérations sont exécutées comme une seule unité. Utilisez les méthodes commit() pour valider les modifications ou rollback() pour annuler.


In [53]:
# Exemple de transaction
try:
    cur.execute("UPDATE utilisateurs SET age = 30 WHERE nom = 'Alice'")
    cur.execute("INSERT INTO utilisateurs (nom, age) VALUES ('Bob', 28)")
    conn.commit()  # Valider les changements
except:
    print("error")
    conn.rollback()  # Annuler en cas d'erreur

#### Indexation
L'indexation améliore les performances des requêtes en accélérant la recherche dans une table. Créez des index sur les colonnes fréquemment utilisées dans les clauses WHERE.


In [56]:
# Exemple de création d'un index
cur.execute("CREATE INDEX idx_nom ON utilisateurs(nom)")
## conn.commit() 

OperationalError: index idx_nom already exists

#### Jointures Avancées
Les jointures permettent de combiner des données de plusieurs tables. Utilisez INNER JOIN, LEFT JOIN, ou RIGHT JOIN selon vos besoins.

In [59]:
# Exemple de jointure
cur.execute("SELECT utilisateurs.nom, commandes.produit FROM utilisateurs INNER JOIN commandes ON utilisateurs.id = commandes.utilisateur_id")

OperationalError: no such table: commandes

#### Sous-requêtes
Les sous-requêtes sont des requêtes imbriquées à l'intérieur d'une autre requête. Elles sont utiles pour effectuer des opérations complexes.


In [ ]:
# Exemple de sous-requête
cur.execute("SELECT nom FROM utilisateurs WHERE id IN (SELECT utilisateur_id FROM commandes WHERE produit = 'Ordinateur')")

#### Utilisation de Paramètres
L'utilisation de paramètres dans les requêtes prévient les attaques par injection SQL et rend le code plus lisible.


In [ ]:
# Exemple avec des paramètres

nom_utilisateur = 'Alice'
cur.execute("SELECT * FROM utilisateurs WHERE nom = ?", (nom_utilisateur,))

#### Travailler avec des Dates
Si vous stockez des dates, utilisez les fonctions de date SQL pour effectuer des opérations.


In [ ]:
# Exemple d'opération avec des dates
cur.execute("SELECT * FROM commandes WHERE date_commande > DATE('2023-01-01')")

### Fermeture de la connexion

N'oubliez pas de fermer la connexion une fois les opérations terminées.

In [10]:
conn.close()